In [2]:
import pandas as pd


In [4]:
Data = pd.read_csv("D:/task/sample_data (1).csv")
Data

,ProductID,ProductName,Category,Price,Rating,ReviewCount,Stock,Discount,Brand,LaunchDate
0,101,Wireless Mouse,Electronics,25.99,4.5,200,150,10%,Logitech,15-01-2022
1,102,Gaming Keyboard,Electronics,75.49,4.7,350,85,5%,Corsair,20-11-2021
2,103,Noise Cancelling Headphones,Electronics,199.99,4.8,125,60,15%,Sony,05-09-2021
3,104,Running Shoes,Sports,49.99,4.3,500,200,20%,Nike,10-02-2022
4,105,Smartwatch,Electronics,159.99,4.6,220,45,10%,Apple,30-03-2022
5,106,Blender,Home & Kitchen,39.99,4.2,180,120,12%,Ninja,25-12-2021
6,107,Coffee Maker,Home & Kitchen,89.99,4.1,240,90,8%,Keurig,15-08-2021
7,108,Yoga Mat,Sports,29.99,4.4,320,300,5%,Manduka,05-01-2022
8,109,Smart TV,Electronics,399.99,4.7,150,25,15%,Samsung,10-06-2022
9,110,Vacuum Cleaner,Home & Kitchen,129.99,4.3,290,70,10%,Dyson,20-10-2021


In [5]:
from pymongo import MongoClient
import os

In [6]:
CSV_PATH = r"D:\task\sample_data (1).csv"  
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "csv_data_db"
COLLECTION_NAME = "products"

In [7]:
# Step 1: Read the CSV file
try:
    df = pd.read_csv(CSV_PATH)
    print(f"CSV loaded successfully with {len(df)} records.")
except Exception as e:
    print("Error reading CSV:", e)
    exit()

# Step 2: Connect to MongoDB
try:
    client = MongoClient(MONGO_URI)
    db = client[DATABASE_NAME]
    collection = db[COLLECTION_NAME]
except Exception as e:
    print("MongoDB connection failed:", e)
    exit()

# Step 3: Convert DataFrame to dictionary format
data_dict = df.to_dict(orient='records')

# Step 4: Insert data into MongoDB
try:
    if data_dict:
        collection.delete_many({})  # Optional: clear previous data
        result = collection.insert_many(data_dict)
        print(f"{len(result.inserted_ids)} documents inserted into '{COLLECTION_NAME}' collection.")
    else:
        print("No data to insert.")
except Exception as e:
    print("Error inserting into MongoDB:", e)
finally:
    client.close()

CSV loaded successfully with 10 records.
10 documents inserted into 'products' collection.


In [10]:
import re


In [ ]:

# Simulated query generation using regex and logic mapping
def generate_mongo_query(user_input):
    # Lowercase for easier parsing
    text = user_input.lower()

    query = {}

    # Handle comparison for numeric fields
    if "price" in text:
        if "greater than" in text or "above" in text:
            match = re.search(r'\$?(\d+)', text)
            if match:
                query["Price"] = { "$gt": float(match.group(1)) }
        elif "less than" in text or "below" in text:
            match = re.search(r'\$?(\d+)', text)
            if match:
                query["Price"] = { "$lt": float(match.group(1)) }

    if "rating" in text:
        if "below" in text:
            match = re.search(r'(\d+(\.\d+)?)', text)
            if match:
                query["Rating"] = { "$lt": float(match.group(1)) }
        elif "4.5 or higher" in text or "above 4.5" in text:
            query["Rating"] = { "$gte": 4.5 }

    if "reviews" in text and "more than" in text:
        match = re.search(r'(\d+)', text)
        if match:
            query["Reviews"] = { "$gt": int(match.group(1)) }

    if "brand" in text:
        brands = re.findall(r"'([^']+)'", user_input)
        if brands:
            query["Brand"] = { "$in": brands }

    if "in stock" in text:
        query["Stock"] = "Yes"

    if "category" in text:
        if "electronics" in text:
            query["Category"] = "Electronics"
        elif "home & kitchen" in text or "home and kitchen" in text:
            query["Category"] = "Home & Kitchen"
        elif "sports" in text:
            query["Category"] = "Sports"

    if "launched after" in text:
        match = re.search(r'after (january 1, 2022|[\w\s,]+)', text)
        if match:
            query["Launch Date"] = { "$gt": "2022-01-01" }

    if "discount" in text:
        match = re.search(r'(\d+)%', text)
        if match:
            query["Discount (%)"] = { "$gte": int(match.group(1)) }

    return {"find": query, "sort": [("Price", -1)] if "descending" in text else []}

# Example interface
if __name__ == "__main__":
    user_input = input("Enter your query: ")
    query_data = generate_mongo_query(user_input)

    print("\nGenerated MongoDB Query:")
    print(f"db.collection.find({query_data['find']})")
    if query_data["sort"]:
        print(f".sort({query_data['sort']})")

In [ ]:
from pymongo import MongoClient
import pandas as pd

In [ ]:


# ---------- CONFIGURATION ----------
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "csv_data_db"
COLLECTION_NAME = "products"
# -----------------------------------

# Step 1: Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client[DATABASE_NAME]
collection = db[COLLECTION_NAME]

# Step 2: Accept or define query
# Example: Replace this with your dynamically generated query
mongo_query = {
    "Rating": {"$lt": 4.5},
    "Reviews": {"$gt": 200},
    "Brand": {"$in": ["Nike", "Sony"]}
}
sort_order = None  # Optional: [("Price", -1)] for descending sort

# Step 3: Execute query
try:
    cursor = collection.find(mongo_query)
    if sort_order:
        cursor = cursor.sort(sort_order)

    results = list(cursor)

    if results:
        # Convert to DataFrame
        df = pd.DataFrame(results)
        
        # Drop MongoDB internal ID if present
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
        
        # Step 4: Choose to Display or Save
        user_choice = input("Do you want to (1) Display or (2) Save as CSV? Enter 1 or 2: ")

        if user_choice == '1':
            print("\nRetrieved Data:")
            print(df.to_string(index=False))
        elif user_choice == '2':
            filename = input("Enter filename to save (e.g., test_case1.csv): ")
            df.to_csv(filename, index=False)
            print(f"\nData saved to {filename}")
        else:
            print("Invalid choice.")
    else:
        print("No matching records found.")

except Exception as e:
    print("Error executing query:", e)
finally:
    client.close()

In [ ]:


# ---------- CONFIGURATION ----------
MONGO_URI = "mongodb://localhost:27017/"
DATABASE_NAME = "csv_data_db"
COLLECTION_NAME = "products"
# -----------------------------------

# Simulated template-based MongoDB query generation
def generate_query(user_input):
    query = {}

    if "rating below" in user_input.lower():
        query["Rating"] = { "$lt": 4.5 }

    if "more than 200 reviews" in user_input.lower():
        query["Reviews"] = { "$gt": 200 }

    if "nike" in user_input.lower() or "sony" in user_input.lower():
        query["Brand"] = { "$in": ["Nike", "Sony"] }

    if "electronics" in user_input.lower():
        query["Category"] = "Electronics"

    if "rating of 4.5 or higher" in user_input.lower():
        query["Rating"] = { "$gte": 4.5 }

    if "in stock" in user_input.lower():
        query["Stock"] = "Yes"

    if "launched after" in user_input.lower():
        query["Launch Date"] = { "$gt": "2022-01-01" }

    if "home & kitchen" in user_input.lower() or "sports" in user_input.lower():
        query["Category"] = { "$in": ["Home & Kitchen", "Sports"] }

    if "discount of 10%" in user_input.lower():
        query["Discount (%)"] = { "$gte": 10 }

    sort_order = [("Price", -1)] if "sorted by price in descending order" in user_input.lower() else None

    return query, sort_order

# Main interactive function
def run_user_interface():
    client = MongoClient(MONGO_URI)
    db = client[DATABASE_NAME]
    collection = db[COLLECTION_NAME]

    try:
        print("\n=== Data Query & Retrieval System ===")
        question = input("Ask a question about your data (e.g., 'Find products with rating below 4.5 and more than 200 reviews from Nike or Sony'):\n")

        # Generate query from input
        mongo_query, sort_order = generate_query(question)

        # Display query
        print("\nGenerated MongoDB Query:")
        print(f"db.collection.find({mongo_query})")
        if sort_order:
            print(f".sort({sort_order})")

        # Run query
        cursor = collection.find(mongo_query)
        if sort_order:
            cursor = cursor.sort(sort_order)

        data = list(cursor)

        if not data:
            print("\nNo matching records found.")
            return

        df = pd.DataFrame(data)
        if "_id" in df.columns:
            df.drop(columns=["_id"], inplace=True)

        # User chooses display or save
        choice = input("\nDo you want to (1) Display or (2) Save to CSV? Enter 1 or 2: ")

        if choice == "1":
            print("\nResult:")
            print(df.to_string(index=False))
        elif choice == "2":
            filename = input("Enter filename (e.g., test_case1.csv): ")
            df.to_csv(filename, index=False)
            print(f"Data saved to '{filename}'")
        else:
            print("Invalid option.")

        # Save query for documentation
        with open("Queries_generated.txt", "a") as f:
            f.write(f"User Question: {question}\nGenerated Query: {mongo_query}\n\n")

    except Exception as e:
        print("Error during processing:", e)
    finally:
        client.close()

# Run
if __name__ == "__main__":
    run_user_interface()